# Miscellaneous Utilities
The follow demonstrates some miscellaneous utilities included in ObsPlus.

## Geodetics
Often it is necessary to calculate geometric parameters (distance, azimuth, etc.) for pairs of entities in two different groups. For example, distance from each event in a catalog to each receiver in an inventory. ObsPlus provides a simple class for doing just that.

In [ ]:
# Load the catalog and inventory from the crandall dataset
import obsplus
from obsplus.utils.geodetics import SpatialCalculator


crandall = obsplus.load_dataset('crandall')
cat = crandall.event_client.get_events()
inv = crandall.station_client.get_stations()

In [ ]:
# init a SpatialCalculator instance (defaults to Earth's params)
spatial_calc = SpatialCalculator()

In [ ]:
# create distance dataframe
df = spatial_calc(entity_1=cat, entity_2=inv)
df.head()

Since a multi-index is used it provides a fairly intuitive way to look up particular event-channel pairs using a tuple of (event_id, seed_id) in conjunction with the `.loc` DataFrame attribute like so:

In [ ]:
event_id = str(cat[0].resource_id)
seed_id = 'UU.MPU..HHZ'

print(df.loc[(event_id, seed_id)])

In [ ]:
# or just get a particular parameter
print(df.loc[(event_id, seed_id), 'azimuth'])

Of course the distances can be converted to km and we can describe the distributions:

In [ ]:
# Convert add km columns and delete m columns
m_columns = [x for x in df.columns if x.endswith('m')]
km_columns = [x.replace('_m', '_km') for x in m_columns]

df_km = (
    df.assign(**{x: df[y] / 1000. for x,y in zip(km_columns, m_columns)}).
    drop(columns=m_columns)
)

# Calculate stats for source reseiver distances
df_km.describe().round(decimals=2)

## Time

Working with dates/times can be frustrating, especially since (as of 2020) ObsPy, numpy, and pandas all use slightly different methods for working with time. ObsPlus provides some utilities to make things a little easier.

In [ ]:
from contextlib import suppress

import numpy as np
import obspy
import obsplus
from obsplus.utils import to_datetime64, to_timedelta64, to_utc

Note that all ObsPlus datafames use numpy/pandas datatypes. 

In [ ]:
df = obsplus.events_to_df(obspy.read_events())

In [ ]:
df['time']

### Converting between time datatypes
We can convert one or all of these to ObsPy `UTCDateTime` objects when needed

In [ ]:
to_utc(df['time'])

### Operations on time
Unlike with obspy's `UTCDateTime`, numpy requires using timedeltas to specify offsets.

In [ ]:
# This raises a TypeError We need to specify units of offset with timedelta64
with suppress(TypeError):
    df['time'] + 1

In [ ]:
df['time'] + np.timedelta64(3600, 's')

In [ ]:
# However, obspy assumes offsets to be in seconds
obspy.UTCDateTime('2020-01-03') + 3600